In [12]:
import pandas as pd
import numpy as np

H1 = [0.5]
H2 = [0.5, 1.0]
H3 = [1.0, 1.5]
H4 = [1.5, 2.0]
H5 = [2.0, 2.5]
H6 = [2.0, 3.0]
H7 = [2.5, 3.5]


# Import Vs30 file
path = 'C:/Users/chia2/Dropbox/URBASIS Paper 2/0.4 Excel/VS30.xlsx'
T = pd.read_excel(path)
# Import the non modified Taxonomy
path2 = 'C:/Users/chia2/Dropbox/URBASIS Paper 2/0.8 OQ/04_03/Esposizione_non_mod.xlsx'
T2 = pd.read_excel(path2)
T_coordinates_tax = T2.iloc[:, 1:3].values
ones_vec = np.zeros((len(T_coordinates_tax), 1))
T_coordinates_tax = np.hstack((T_coordinates_tax, ones_vec))
T_coordinates_vs = T.iloc[:, 0:3].values
Taxonomy = T2.iloc[:, 3].values

# Update Taxonomy for LFI and LFM
for i in range(len(Taxonomy)):
    st = str(Taxonomy[i])
    if st[3:6] == 'LFI':
        sz = st[0:12]
        sx = st[-3:]
        sm = sz + sx
        Taxonomy[i] = sm
    elif st[3:6] == 'LFM':
        sz = st[0:10]
        sx = st[-3:]
        sm = sz + sx
        Taxonomy[i] = sm

# Convert CDL to DUL and vice versa
# This is to convert the so calculated fragility frunction to an exposure that is defined for a slightly different exposure

for i in range(len(Taxonomy)):
    st = str(Taxonomy[i])
    if st[3:6] == 'LFI':
        sz = st[9:12]
        if sz == 'CDL':
            st = st[:9] + 'DUL' + st[12:]
        elif sz == 'CDM':
            st = st[:9] + 'DUM' + st[12:]
        elif sz == 'CDH':
            st = st[:9] + 'DUH' + st[12:]
        else:
            st = st[:9] + 'DUL' + st[12:] + '_SOS'
        Taxonomy[i] = st
    elif st[3:6] == 'LFM':
        sz = st[7:10]
        if sz == 'CDL':
            st = st[:7] + 'DUL' + st[10:]
        elif sz == 'CDM':
            st = st[:7] + 'DUM' + st[10:]
        elif sz == 'CDH':
            st = st[:7] + 'DUH' + st[10:]
        else:
            st = st[:7] + 'DNO' + st[10:] + '_SOS'
        Taxonomy[i] = st

tol1 = 0.002

# Find matching coordinates
for i in range(len(T_coordinates_tax)):
    for j in range(len(T_coordinates_vs)):
        if abs(T_coordinates_vs[j, 0] - T_coordinates_tax[i, 0]) < tol1 and abs(T_coordinates_vs[j, 1] - T_coordinates_tax[i, 1]) < tol2:
            T_coordinates_tax[i, 2] = T_coordinates_vs[j, 2]       
            
            
# Update tolerances if needed
tol1 = 0.005
tol2 = 0.005

# Find matching coordinates with updated tolerances
for i in range(len(T_coordinates_tax)):
    if T_coordinates_tax[i, 2] == 0:
        for j in range(len(T_coordinates_vs)):
            if abs(T_coordinates_vs[j, 0] - T_coordinates_tax[i, 0]) < tol1 and abs(T_coordinates_vs[j, 1] - T_coordinates_tax[i, 1]) < tol2:
                T_coordinates_tax[i, 2] = T_coordinates_vs[j, 2]

# Update tolerances
tol1 = 0.006
tol2 = 0.006

# Find matching coordinates with updated tolerances
for i in range(len(T_coordinates_tax)):
    if T_coordinates_tax[i, 2] == 0:
        for j in range(len(T_coordinates_vs)):
            if abs(T_coordinates_vs[j, 0] - T_coordinates_tax[i, 0]) < tol1 and abs(T_coordinates_vs[j, 1] - T_coordinates_tax[i, 1]) < tol2:
                T_coordinates_tax[i, 2] = T_coordinates_vs[j, 2]

# Update tolerances
tol1 = 0.007
tol2 = 0.007

# Find matching coordinates with updated tolerances
for i in range(len(T_coordinates_tax)):
    if T_coordinates_tax[i, 2] == 0:
        for j in range(len(T_coordinates_vs)):
            if abs(T_coordinates_vs[j, 0] - T_coordinates_tax[i, 0]) < tol1 and abs(T_coordinates_vs[j, 1] - T_coordinates_tax[i, 1]) < tol2:
                T_coordinates_tax[i, 2] = T_coordinates_vs[j, 2]

# Create a table with updated coordinates
Tabella = pd.DataFrame(T_coordinates_tax)
Tabella.columns = ['X', 'Y', 'Z']
Tabella.to_csv('new_tax.csv', index=False)

# Update Vs values based on Z coordinates
Vs = np.zeros(len(T_coordinates_tax))
for i in range(len(T_coordinates_tax)):
    if T_coordinates_tax[i, 2] < 180:
        Vs[i] = 150
    elif T_coordinates_tax[i, 2] >= 180 and T_coordinates_tax[i, 2] < 220:
        Vs[i] = 180
    elif T_coordinates_tax[i, 2] >= 220 and T_coordinates_tax[i, 2] < 270:
        Vs[i] = 250
    elif T_coordinates_tax[i, 2] >= 270 and T_coordinates_tax[i, 2] < 320:
        Vs[i] = 300
    elif T_coordinates_tax[i, 2] >= 320 and T_coordinates_tax[i, 2] < 380:
        Vs[i] = 360
    elif T_coordinates_tax[i, 2] >= 380 and T_coordinates_tax[i, 2] < 430:
        Vs[i] = 400
    elif T_coordinates_tax[i, 2] >= 430 and T_coordinates_tax[i, 2] < 480:
        Vs[i] = 450
    elif T_coordinates_tax[i, 2] >= 480:
        Vs[i] = 800

Vs = Vs.reshape(-1, 1)
#Taxonomy_new = np.char.add(Taxonomy, '_' + np.char.mod('%d', Vs.flatten()))
Taxonomy_new = [tax + '_' + str(v[0]) for tax, v in zip(Taxonomy, Vs)]

# Update heights in Taxonomy
for i in range(len(Taxonomy_new)):
    st = str(Taxonomy[i])
    if st[-1] == "1" or st[-5:] == "1_SOS":
        H_str = str(max(H1))
    elif st[-1] == "2" or st[-5:] == "2_SOS":
        H_str = str(max(H2))
    elif st[-1] == "3" or st[-5:] == "3_SOS":
        H_str = str(max(H3))
    elif st[-1] == "4" or st[-5:] == "4_SOS":
        H_str = str(max(H4))
    elif st[-1] == "5" or st[-5:] == "5_SOS":
        H_str = str(max(H5))
    elif st[-1] == "6" or st[-5:] == "6_SOS":
        H_str = str(max(H6))
    else:
        H_str = str(max(H7))
    Taxonomy_new[i] = Taxonomy_new[i] + '_' + H_str

T2['Taxonomy'] = Taxonomy_new
T2.to_csv('exposure_mod26_06.csv', index=False)            
            